# Upload SVD inputs and outputs to database
Natalia Vélez, November 2020

In [1]:
import pymongo
import os
import bson
import json
import pickle
import gridfs
import pandas as pd
import numpy as np
from os.path import join as opj
from tqdm import notebook

Connect to database:

In [2]:
keyfile = 'credentials.key'
creds = open(keyfile, "r").read().splitlines()
myclient = pymongo.MongoClient('134.76.24.75', username=creds[0], password=creds[1], authSource='ohol') 
db = myclient.ohol

print(db)
print(db.list_collection_names())

Database(MongoClient(host=['134.76.24.75:27017'], document_class=dict, tz_aware=False, connect=True, authsource='ohol'), 'ohol')
['lifelogs', 'families', 'test']


Load SVD data:

In [3]:
U = np.loadtxt('../3_technology/outputs/svd/U_hat.txt')
s = np.loadtxt('../3_technology/outputs/svd/s_hat.txt')
V = np.loadtxt('../3_technology/outputs/svd/Vh_hat.txt')
avatars = np.loadtxt('../3_technology/outputs/svd/input_playerIDs.txt')
items = np.loadtxt('../3_technology/outputs/activity_features.txt')

Record inputs

Assemble record:

In [6]:
svd_record = {}

# Serialize 1d entries
svd_record['s'] = list(s)

avatar_out = avatars.astype(int)
svd_record['avatars'] = list(avatars)

#item_out = items.astype(int)
svd_record['items'] = list(items)

# Pickle 2d entries
U_bin = bson.binary.Binary(pickle.dumps(U, protocol=2), subtype=128)
V_bin = bson.binary.Binary(pickle.dumps(U, protocol=2), subtype=128)

Save large files using GridFS instance:

In [7]:
fs = gridfs.GridFS(db)
U_id = fs.put(U_bin, filename='svd_U')
V_id = fs.put(V_bin, filename='svd_V')
print(U_id)
print(V_id)

5fa17b3de45a79704dad77ac
5fa17f19e45a79704dad85b3


In [10]:
print(db.list_collection_names())

['families', 'test', 'lifelogs', 'fs.files', 'svd', 'fs.chunks']


In [12]:
db.fs.chunks

Collection(Database(MongoClient(host=['134.76.24.75:27017'], document_class=dict, tz_aware=False, connect=True, authsource='ohol'), 'ohol'), 'fs.chunks')

Save everything else to a collection:

In [8]:
svd_col = db.svd
svd_col.drop()

In [9]:
svd_col = db.svd
svd_col.insert_one(svd_record)